In [1]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])
    
for subdir in ["tools"] + glob.glob("./sem*"):
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        


tools ['tools/set_up_magics.ipynb']
[NbConvertApp] Converting notebook tools/set_up_magics.ipynb to markdown
[NbConvertApp] Writing 22957 bytes to tools/README.md
./sem04-asm-arm ['./sem04-asm-arm/arm.ipynb']
[NbConvertApp] Converting notebook ./sem04-asm-arm/arm.ipynb to markdown
[NbConvertApp] Writing 7422 bytes to ./sem04-asm-arm/README.md
./sem11-mmap-instrumentation ['./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb']
[NbConvertApp] Converting notebook ./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to markdown
[NbConvertApp] Writing 23442 bytes to ./sem11-mmap-instrumentation/README.md
./sem15-ptrace ['./sem15-ptrace/ptrace.ipynb']
[NbConvertApp] Converting notebook ./sem15-ptrace/ptrace.ipynb to markdown
[NbConvertApp] Writing 15768 bytes to ./sem15-ptrace/README.md
./sem10-file-attributes ['./sem10-file-attributes/file-attrib.ipynb']
[NbConvertApp] Converting notebook ./sem10-file-attributes/file-attrib.ipynb to markdown
[NbConvertApp] Writing 16319 bytes

In [2]:
" ".join(['jupyter', 'nbconvert', './sem04-asm-arm/arm.ipynb', '--to', 'pdf', '--output', 'arm'])

'jupyter nbconvert ./sem04-asm-arm/arm.ipynb --to pdf --output arm'

In [5]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        try:
            fname = os.path.join(path, matchobj.group(1))
            if not fname.endswith("__FILE__"):
                with open(fname, "r") as f:
                    return " ``` \n " + f.read() + "\n ``` \n"
        except:
            return ""
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [6]:
for sfx in [".ipynb", ".md"]:
    for fname in glob.glob("./sem*/*" + sfx) + glob.glob("./tools/*" + sfx):
        improve_file(fname)

dos2unix: converting file ./sem04-asm-arm/arm.ipynb to Unix format ...
dos2unix: converting file ./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to Unix format ...
dos2unix: converting file ./sem15-ptrace/ptrace.ipynb to Unix format ...
dos2unix: converting file ./sem10-file-attributes/file-attrib.ipynb to Unix format ...
dos2unix: converting file ./sem03-ints-floats/sem3_floats.ipynb to Unix format ...
dos2unix: converting file ./sem03-ints-floats/sem3_ints.ipynb to Unix format ...
dos2unix: converting file ./sem07-asm-x86-x87-sse/sse_x86.ipynb to Unix format ...
dos2unix: converting file ./sem07-asm-x86-x87-sse/floating_point_x86.ipynb to Unix format ...
dos2unix: converting file ./sem14-fifo-proc/fifo-proc.ipynb to Unix format ...
dos2unix: converting file ./sem12-fork-exec-pipe/fork-pipe-exec.ipynb to Unix format ...
dos2unix: converting file ./sem08-asm-x86-nostdlib/nostdlib.ipynb to Unix format ...
dos2unix: converting file ./sem13-signal/signal.ipynb to Unix format .

In [ ]:
for subdir in glob.glob("./sem*") + glob.glob("./tools"):
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        get_ipython().system("git add {}/*{}".format(os.path.basename(subdir), sfx))
        get_ipython().system("git add -f {}/*{}".format(os.path.basename(subdir)))
get_ipython().system("git add -u")
get_ipython().system("git commit -m 'yet another update'")
get_ipython().system("git push origin master")